In [0]:
%pip install azure-mgmt-datafactory azure-identity

import os
import time
from azure.identity import DefaultAzureCredential
from azure.mgmt.datafactory import DataFactoryManagementClient

In [0]:
# set env variable
ENV_NAME = os.getenv('ENV_NAME','')
spark.sql('SET var.ENV_NAME = '+ ENV_NAME)

SUBSCRIPTION_NAME = dbutils.secrets.get(scope=f"dataplatform{ENV_NAME}-dbss", key="subscription-id")
RESOURCE_GROUP_NAME = f"rg-dataplatform-{ENV_NAME}"
DATA_FACTORY_NAME = f"adf-{ENV_NAME}"
UC_CREDENTIAL_NAME = f"dataplatform{ENV_NAME}_azservices-dbsc"

# Recieve ADF pipeline name passed in through parameter
PIPELINE_NAME = dbutils.widgets.get("p_pipeline_name")

In [0]:
credential=dbutils.credentials.getServiceCredentialsProvider(UC_CREDENTIAL_NAME)

adf_client = DataFactoryManagementClient(credential=credential, subscription_id=SUBSCRIPTION_NAME)

run_response = adf_client.pipelines.create_run(
    resource_group_name=RESOURCE_GROUP_NAME,
    factory_name=DATA_FACTORY_NAME,
    pipeline_name=PIPELINE_NAME
)

# Poll status
run_id = run_response.run_id
print(f"Pipeline run initiated: {run_id}")

while True:
    pipeline_run = adf_client.pipeline_runs.get(
        RESOURCE_GROUP_NAME,
        DATA_FACTORY_NAME,
        run_id
    )
    status = pipeline_run.status
    print(f"ADF pipeline status: {status}")
    
    if status in ("Succeeded", "Failed", "Cancelled"):
        break
    time.sleep(10)

if status != "Succeeded":
    raise Exception(f"ADF pipeline failed with status: {status}")